In [1]:
import pandas as pd

df = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "description": [
        "A red apple that is juicy and sweet.",
        "A yellow banana full of potassium.",
        "A sports car with a powerful engine.",
        "A mountain bike built for rough terrain."
    ]
})


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")  # small, fast model
embeddings = model.encode(df["description"].tolist(), convert_to_numpy=True)


/media/sandeep/363843093842C7971/project_ITM_samsher/My_Startup_Projects/qdrant/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from qdrant_client import QdrantClient

client = QdrantClient(host="localhost", port=6333)

client.recreate_collection(
    collection_name="semantic_text_search",
    vectors_config={"size": embeddings.shape[1], "distance": "Cosine"}
)

client.upload_collection(
    collection_name="semantic_text_search",
    vectors=embeddings.tolist(),
    payload=df.to_dict(orient="records"),
    ids=df["id"].tolist()
)


/tmp/ipykernel_177046/297415334.py:5: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [5]:
query = "a vehicle designed for speed and performance"
query_vector = model.encode(query).tolist()

results = client.search(
    collection_name="semantic_text_search",
    query_vector=query_vector,
    limit=3
)

for hit in results:
    print(f"ID: {hit.id} | Desc: {hit.payload['description']} | Score: {hit.score:.4f}")


/tmp/ipykernel_177046/1271877106.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


ID: 3 | Desc: A sports car with a powerful engine. | Score: 0.5779
ID: 4 | Desc: A mountain bike built for rough terrain. | Score: 0.4027
ID: 1 | Desc: A red apple that is juicy and sweet. | Score: 0.1524
